# Hamblin semantics introductory fragment
## Author: Kyle Rawlins
### Updated: 3/10/24

This is an implementation of compositional Hamblin semantics.  It is currently quite incomplete, but walks through the basic principles.  It also is useful for introducing some lexicon manipulation techniques in the lambda notebook (see also the appendix).

Hamblin's influential idea is that, if the denotation of root declarative clause is a proposition, then the denotation of an interrogative can be thought of as a set of propositions, each corresponding to an answer to the question that interrogative would ask. For example, a polar interrogative "Is it raining?" would denote a set of two alternative propositions: the proposition that it is raining, and the proposition that it isn't. (See Hamblin 1973: [Questions in Montague English](https://www.jstor.org/stable/25000703), *Foundations of Language* 10.)

A compositional Hamblin semantics is one that pushes this idea a bit further. Not only are clause denotations lifted to sets, any denotation during composition is lifted to a set. For ordinary cases, these sets may be singleton, but certain elements may then introduce or manipulate alternatives, leading to both question meanings, but also other compositional interactions of alternatives and operators. This idea was most famously developed in a 2002 paper by Kratzer and Shimoyama, [Indeterminate Pronouns: The View from Japanese](https://people.umass.edu/partee/RGGU_2004/Indeterminate%20Pronouns.pdf).

In [ ]:
%lambctl reset

In [ ]:
# set up a type system with an intensional type
type_e = types.type_e
type_t = types.type_t
type_n = types.type_n
type_s = types.BasicType("s")
ts = meta.get_type_system()
ts.add_atomic(type_s)
ts

## Basics: singleton set denotations

As noted above, one core idea is that "ordinary" denotations are lifted to be singleton sets. When writing out a lexicon, therefore, where you previously would have written a regular element of some type $\alpha$, you will now write a singleton set containing that element, which in the lambda notebook's type system is indicated by $\{\alpha\}$. (In some approaches, this is alternatively modeled by using characteristic functions instead of sets, but here I take the direct implementation.)

One could just do this manually, e.g. writing a denotation as a set: (n.b. the inner parentheses are required)

In [ ]:
%lamb ||cat|| = {(L x_e : L w_s : Cat(w,x))}

However, in the present setting we can do this systematically. Essentially, to "hamblinize" the lexicon, we want to give a general procedure for converting ordinary meanings into singleton sets.

The following cell defines a python function that will convert an arbitrary typed expression into a set. It then defines a transform that can be used when writing a lexical entry that applies the hamblinization operation.

In [ ]:
hamblinize_unit = %te L x_X : {x}
hamblinize_unit

In [ ]:
hamblinize_unit(te("L x: Cat_<e,t>(x)")).reduce_all()

We can apply an operation like this when building a lexical item by setting an "equality transform":

In [ ]:
lamb.parsing.eq_transforms["hamblin"] = hamblinize_unit

Let's see this in operation:

In [ ]:
%%lamb
||cat|| =<hamblin> L x_e : L w_s : Cat(w,x)
||gray|| =<hamblin> L x_e : L w_s : Gray(w,x)
||john|| =<hamblin> John_e
x =<hamblin> L y_e : y
||test|| = L x_e : Test(x) # don't hamblinize this

## Basics: Pointwise Function Application

Ordinary Function Application will not work on these sorts of lexical items, because they are not themselves functions. However, they contain functions. The standard approach to generalizing FA is to interpret it *pointwise* -- given a set of functions and arguments of the right types, apply each argument to each function and return the resulting set. I provide two versions of this operation below.

First, we can look at the most general implementation of Pointwise Function Application (PFA), which is written using a combinator. The types `X` and `Y` are variables over arbitrary types.

(This version of PFA characteristically results in complex but simplifiable expressions, that the lambda notebook doesn't itself simplify. In the long run, one would want to automatically simplify these expressions; but this would require a theorem prover that is currently beyond the scope of this project.)

In [ ]:
pfa_combinator = %te L f_{<X,Y>} : L a_{X} : Set x_Y : Exists f1_<X,Y> : (Exists a1_X : (f1 << f & a1 << a) & x <=> f1(a1))
pfa_combinator

In [ ]:
system = lang.td_system.copy()
system.add_binary_rule(pfa_combinator, "PFA")
lang.set_system(system)
system

In [ ]:
john * cat

Formulas of this style are pretty annoying to work with, and in practice most Hamblin Semantics examples work with listed sets, not sets determined by arbitrary predicates. The lambda notebook has a listed set implementation (the class `ListedSet`), and so for the simplified version of PFA I'll add support for this. The second version of PFA leaves any listed sets as listed sets, rather than lifting them to generalized sets. Essentially we special-case the PFA operation.

Note that some limited reduction does occur with the $\in$ operator; $x \in \{y \:|\: \phi\}$ gets converted to $(\lambda y : \phi)(x)$ and reduced, illustrated in the next cell.

In [ ]:
r = te("x_e << (Set y_e : Test_<e,t>(y))")
r.simplify_all().derivation

In [ ]:
def pfa_listed(fun, arg):
    result = list()
    # build every function-arg combination
    for felem in fun.args:
        for aelem in arg.args:
            result.append(felem(aelem))
    return meta.sets.sset(result)

# generalized PFA: use the listed set code if it can work, otherwise use the more general combinator
def pfa_general(fun, arg):
    ts = meta.get_type_system()
    general = pfa_combinator(fun)(arg) # do this for type checking
    if isinstance(fun, meta.sets.ListedSet) and isinstance(arg, meta.sets.ListedSet):
        return pfa_listed(fun, arg)
    else:
        return general.reduce_all()
    
system = lang.td_system.copy()
system.add_binary_rule_uncurried(pfa_general, "PFA")
lang.set_system(system)
system

In [ ]:
john * cat

## Denotations for interrogatives

The above code has no alternative-sensitive denotations, so essentially reconstructs an ordinary compositional semantics with added set wrappers. This by itself is a bit pointless, but what we'll do next is add some alternative-sensitive items to the lexicon. In particular, the denotation of an interrogative pronoun like "who" in this system is treated as a set of individuals. The most general version of this would look something like ${x_e \:|\: Human(x)}$. However, for a more general introduction, we will use a listed set that fixes a specific domain.

In [ ]:
%%lamb
## To simplify, let's take there to only be three human-like entities in the universe.
||who|| = {John_e, Mary_e, Sue_e}
||left|| =<hamblin> L x_e : L w_s : Left(w,x)
||saw|| =<hamblin> L x_e : L y_e : L w_s : Saw(w,y,x)

Now, the power of PFA is a bit more realized. The verb is still singleton, but it can combine with interrogative pronouns to produce non-singleton sets of alternatives. Intuitively, each of these alternative propositions is taken to correspond to a possible answer:

In [ ]:
who * left

In the first-pass analysis for English, we interpret interrogative pronouns in-situ (their corresponding position in some other languages):

In [ ]:
(saw * who) # VP denotation

What happens with this can be described at an intuitive level as the alternatives "percolating" up the tree. Here's a fully composed sentence denotation with an object-position "who":

In [ ]:
john * (saw * who)

In [ ]:
(john * (saw * who)).tree()

Multiple interrogatives are relative straighforward on this set of assumptions:

In [ ]:
who * (saw * who)

In [ ]:
(who * (saw * who)).tree()

## Existentials and free choice

So far, we have essentially reconstructed Hamblin's original semantics. Kratzer and Shimoyama propose that this system can also be used to account for existentials in Japanese, but treating them as consisting of an operator that takes scope, and an in-situ "indeterminate" pronoun that has the semantics illustrated above. In subsequent literature, this has been used to account for various "free choice" indefinites in many languages. Here I'll just present the core idea, using English lexical items.

What we need to add is an alternative-aware operator that quantifies over set elements. Here are two possibilities from Kratzer and Shimoyama:

In [ ]:
%%lamb
||HExists|| = L p_{<s,t>} : {(Lambda w_s  : Exists q_<s,t> : q(w) & (q << p))}
||HForall|| = L p_{<s,t>} : {(Lambda w_s  : Forall q_<s,t> : (q << p) >> q(w))}

These operators compose via ordinary FA, and intuitively collapse alternative structure, resulting in a singleton denotation.

In [ ]:
HExists * (who * (saw * john))

In [ ]:
HExists * (john * (saw * who))

In [ ]:
(HExists * (john * (saw * who))).tree()

Various free choice items can also be analyzed using these techniques. I'll introduce the idea using English "any", though this is not probably the best candidate item for this kind of analysis.

A puzzle about "any" is that it intuitively seems universal at some level, and in some contexts patterns as universal, but in others (e.g. in the scope of negation) it appears to be existential/indefinite. (There's a large literature on this that I'm not representing here.) A Hamblin account of such an item would say that it introduces alternatives like many other indefinites, but associates with a universal, not an existential operator. This doesn't by itself solve the licensing problem for "any" (it generally does require certain kinds of licensing contexts, e.g. negation, generics, etc), but does get the right interpretation across contexts. Here's a sample for a simple negative sentence:

In [ ]:
%%lamb
||anyone|| = {John_e, Mary_e, Sue_e}
||neg|| =<hamblin> L p_<s,t> : L w_s : ~p(w) # non-alternative-sensitive negation

In [ ]:
neg * (john * (saw * anyone))

In [ ]:
HForall * (neg * (john * (saw * anyone)))

## The general version

Going back to the general version of the system, we don't want to actually define "who" extensionally in the general case. Here is how things would go with a more general implementation.

In [ ]:
%%lamb
||who|| = Set x_e : Human(x)

As noted above, at the moment the lambda notebook doesn't do much simplification of these expressions, so they will be a bit unwieldy.

In [ ]:
(cat * who).tree()

In [ ]:
john * (saw * who)

In [ ]:
who * (saw * john)

In [ ]:
HExists * (who * (saw * john))

That's it for this notebook! There's lots more one can do with a compositional Hamblin semantics, and this idea has been the jumping off point for accounts of free choice, indefinites, and various complexities of interrogative semantics, as well as counter-proposals like inquisitive semantics.

### Appendix: another technique for Hamblinization

Another way of Hamblinizing a lexicon would be to write extra magics for converting whole lexicons at once.  Here's a sketch of how to do this.

In [ ]:
def hamblinize_item(item):
    """Hamblinize a single lexical item.  Do so by building a singleton set out of it."""
    if meta.ts_compatible(item.type, meta.tp("{?}")):
        return item # assume that a set denotation is already Hamblinized
    elif meta.ts_compatible(item.type, meta.tp("<{?},?>")):
        return item # Assume that a function whose domain is a set is an alternative-aware operator and shouldn't be modified

    # build a new lang.Item object that wraps the content as a singleton set
    new_i = item.copy()
    new_i.content = meta.sets.sset([item.content])
    return new_i

# in the following two magics, only object language definitions are affected
def h_magic(self, accum):
    """Hamblinize the accumulated definitions from a single cell, as a post-processing step"""

    new_accum = lamb.magics.process_items(hamblinize_item, accum)[0]
    self.env.update(new_accum)
    return new_accum

def h_magic_env(self):
    """Hamblinize an entire env"""

    self.env.update(lamb.magics.process_items(hamblinize_item, self.env)[0]) # hamblinize every variable
    self.shell.push(dict(self.env)) # export the new variables to the interactive shell
    return lamb.parsing.html_output(self.env)

lamb.magics.LambMagics.specials_post["hamblinize"] = h_magic
lamb.magics.LambMagics.specials["hamblinize_all"] = h_magic_env

type_s = types.BasicType("s")
ts = meta.get_type_system()
ts.add_atomic(type_s)
ts

In [ ]:
%%lamb reset,hamblinize
||cat|| = L x_e : L w_s : Cat(w,x)
||gray|| = L x_e : L w_s : Gray(w,x)
||john|| = J_e
x = L y_e : y

In [ ]:
%%lamb
||test|| = L x_e : Test(x)

In [ ]:
# lifts any previous un-Hamblinized items to Hamblin sets
%lambctl hamblinize_all